<a id="Table"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Table of Contents
</h1>
</div>

 | S.No       |                   Heading                |
 | :------------- | :-------------------:                |                     
 |  01 |  [**Libraries**](#library)                        |  
 |  02 |  [**JSON to Pandas CSV**](#json)                |
 |  03 |  [**Sample Images**](#sample-images)      |
 |  04 |  [**Config**](#config)                |
 |  05 |  [**Custom Dataset**](#dataset)  |
 |  06 |  [**Image Augmentation : Albumentations**](#Transform)   |
 |  07 |  [**Create Dataloader**](#dataloader)   |
 |  08 |  [**Model**](#model) |
 |  09 |  [**Trainer⚡**](#Trainer) |
 |  10 |  [**Plot Metrics**](#metrics) |

## **<span style="color:#648c54;">Identify plant species of the Americas from herbarium specimens</span>**

The FGVC9 2022 Herbarium Challenge is to identify melastome species from herbarium specimens provided by the New York Botanical Garden (NYBG).
Background

There are more than 400,000 known plant species with an estimated 80,000 still to be discovered. In flowering plants, it takes approximately 35 years from plant collection to species description while less than 16% of new species are described in less than 5 years. It has also been suggested that ‘herbaria are a major frontier for species’ with more than 50% of unknown species already in herbarium collections.

The dataset encompasses over 90% of all plant taxa in North America, spanning 1.05M images for 15501 plant taxa, collected from 60 different institutions around the world.

References : 
* https://www.kaggle.com/c/herbarium-2022-fgvc9/discussion/307792
* https://www.kaggle.com/c/herbarium-2019-fgvc6

<a id="library"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Libraries
</h1>
</div>

In [ ]:
import os
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import json
import cv2
import seaborn as sns
from skimage import io

!pip install albumentations
!pip install pytorch-lightning
!pip install timm
!pip install torchmetrics

import timm
import torch
import torch.nn as nn
import torch.functional as F
from torchvision import datasets, transforms,models
from torch.utils.data import Dataset,DataLoader

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import torchmetrics


import albumentations as A
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2

import pytorch_lightning as pl
%matplotlib inline

<a id="json"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> JSON to Pandas CSV
</h1>
</div>

**Reference/Credits** :  [JSON -> PANDAS📊
@Sanskar Hasija](https://www.kaggle.com/odins0n/json-pandas-herbarium-2022/)

In [ ]:


TRAIN_DIR = "../input/herbarium-2022-fgvc9/train_images/"
TEST_DIR = "../input/herbarium-2022-fgvc9/test_images/"

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)
    
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [TRAIN_DIR + image["file_name"] for image in train_meta["images"]]
category_ids = [annot["category_id"] for annot in train_meta["annotations"]]
genus_ids = [annot["genus_id"] for annot in train_meta["annotations"] ]
test_ids = [image["image_id"] for image in test_meta]
test_dirs = [TEST_DIR + image["file_name"] for image in test_meta ]

train = pd.DataFrame(data =np.array([image_ids , image_dirs, genus_ids, category_ids ]).T, 
                     columns = ["image_id", "directory","genus_id", "category",])
test = pd.DataFrame(data =np.array([test_ids  , test_dirs ]).T, 
                    columns = ["image_id", "directory",])

train.to_csv("train.csv", index = False)
test.to_csv("test.csv", index = False)

In [ ]:
df_train = pd.read_csv('train.csv')
genera = pd.DataFrame(train_meta['genera'])
genera  = df_train.merge(genera,on='genus_id')
genera.head()

In [ ]:
df_train.category.nunique()

Total No. Of Unique Classes/Labels : **15501**

## **<span style="color:#648c54;">Top 10 Genera Distribution</span>**

In [ ]:
genera['genus'].value_counts().head(10).plot(kind='bar')

<a id="sample-images"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Sample Images 
</h1>
</div>

In [ ]:
from PIL import Image
def display_images(genus):
    images = df_train.loc[genera['genus'] == genus]['directory'][:12]
    i = 1
    fig = plt.figure(figsize = (10, 10))
    plt.suptitle(genus, fontsize = '35')
    for image in images:
        ax = fig.add_subplot(3, 4, i)
        image = Image.open(image).convert('RGB')
        ax.imshow(image)
        i+=1
    fig.tight_layout()
    plt.show()

display_images('Carex')

In [ ]:
display_images('Astragalus')

## **<span style="color:#648c54;">Encode Class Labels</span>**

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df_train['category'])
df_train['category'] = le.transform(df_train['category'])
df_train.category.nunique()

Since the Training Data is large approx 119 GB will take a lot time in training processs,We will work with a smaller sample of first 5000 images.

In [ ]:
df_train = df_train[:5000]

<a id="config"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Config 
</h1>
</div>

In [ ]:

class CFG:
    seed = 42
    model_name = 'resnet50'
    pretrained = True
    img_size = 224
    num_classes = df_train.category.nunique() # 86 - since the train set is reduced to 5000 images 
    lr = 5e-4
    min_lr = 1e-6
    t_max = 20
    num_epochs = 20
    batch_size = 32
    accum = 1
    precision = 16
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
seed_everything(CFG.seed)

In [ ]:
df_train.head()

<a id="dataset"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Custom Dataset 
</h1>
</div>

In [ ]:
class HerbariumDataset(Dataset):
    def __init__(self,data,transform=None):
        self.annotations = data
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        
        img_path = self.annotations.iloc[index,1]
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,3]))
        
        if self.transform:
            image = self.transform(image=np.array(image))
        
        return(image,y_label) 

<a id="Transform"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Image Augmentation : Albumentations 
</h1>
</div>

**Image augmentation** is a process of creating new training examples from the existing ones. To make a new sample, you slightly change the original image. For instance, you could make a new image a little brighter; you could cut a piece from the original image; you could make a new image by mirroring the original one, etc.
![](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Falbumentations.ai%2Fdocs%2Fimages%2Fintroduction%2Fimage_augmentation%2Faugmentation.jpg&f=1&nofb=1)

**Albumentations** is a Python library for fast and flexible image augmentations. Albumentations efficiently implements a rich variety of image transform operations that are optimized for performance, and does so while providing a concise, yet powerful image augmentation interface for different computer vision tasks, including object classification, segmentation, and detection.

To learn more [**Albumentations**](https://albumentations.ai/docs/)

In [ ]:
def Transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(),
            ToTensorV2(),
        ])

<a id="dataloader"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Create Dataloader 
</h1>
</div>

In [ ]:
train_dataset = HerbariumDataset(df_train,transform=Transform('train'))
valid_dataset = HerbariumDataset(df_train,transform=Transform('valid'))

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

## **<span style="color:#648c54;">Transformed Image</span>**

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images['image'][1].shape)
print(labels[1].shape)
plt.imshow(images['image'][3].squeeze().permute(1,2,0), cmap='Greys_r')

<a id="model"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Model
</h1>
</div>

In [ ]:
class CustomResNet(nn.Module):
    def __init__(self, model_name='resnet18', pretrained=False):
        super().__init__()
        # timm contains collection of pretrained image models like torchvision.models
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
        self.model.fc = nn.Linear(in_features, CFG.num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class LitHerbarium(pl.LightningModule):
    def __init__(self, model):
        super(LitHerbarium, self).__init__()
        self.model = model # ResNet50 
        self.metric = torchmetrics.F1(num_classes=CFG.num_classes) # F1 Score
        self.criterion = nn.CrossEntropyLoss() # Cross Entropy loss
        self.lr = CFG.lr # Learning Rate

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    def configure_optimizers(self):
        """Choose what optimizers and learning-rate schedulers to use in your optimization"""
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=CFG.t_max, eta_min=CFG.min_lr)

        return {'optimizer': self.optimizer, 'lr_scheduler': self.scheduler}

    def training_step(self, batch, batch_idx):
        """Here you compute and return the training loss and some additional metrics for e.g. the progress bar or logger"""
        image,target = batch
        image = image['image']
        output = self.model(image)
        loss = self.criterion(output, target)
        score = self.metric(output.argmax(1), target)
        logs = {'train_loss': loss, 'train_f1': score, 'lr': self.optimizer.param_groups[0]['lr']}
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        """Operates on a single batch of data from the validation set .calculate anything of interest like accuracy."""
        image,target = batch
        image = image['image']
        output = self.model(image)
        loss = self.criterion(output, target)
        score = self.metric(output.argmax(1), target)
        logs = {'valid_loss': loss, 'valid_f1': score}
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

#### **To Learn more about** [LightningModule](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html?highlight=training_step#)

In [ ]:
model = CustomResNet(model_name=CFG.model_name, pretrained=CFG.pretrained)
lit_model = LitHerbarium(model)
lit_model

In [ ]:
# pytorch computational Graph of ResNet50 

# # !pip install torchviz
# from torchviz import make_dot
# batch = next(iter(train_loader))
# images , labels = batch
# label = model(images['image'])
# make_dot(label, params=dict(list(lit_model.named_parameters())))

<a id="Trainer"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Trainer⚡
</h1>
</div>

* **Max_epochs** : Stop training once this number of epochs is reached
* **gpus** : Number of GPUs to train on (int)
* **accumulate_grad_batches** : Accumulates grads every k batches or as set up in the dict. Trainer also calls optimizer.step() for the last indivisible step number.
* **precision** : 16 bit floating points to reduce memory footprint during model training. This can result in improved performance.
* **callbacks** : A callback is a self-contained program that can be reused across projects.
* **checkpoint_callback / enable_checkpointing** : If True, enable checkpointing Else None.
* **logger** :  PyTorch TensorBoard logging under the hood, and stores the logs to a directory 

To learn more about [**Trainer**](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.trainer.trainer.Trainer.html?highlight=trainer)

In [ ]:
logger = CSVLogger(save_dir='logs/', name=CFG.model_name)
logger.log_hyperparams(CFG.__dict__)
checkpoint_callback = ModelCheckpoint(monitor='valid_loss',
                                      save_top_k=1,
                                      save_last=True,
                                      save_weights_only=True,
                                      filename='checkpoint/{epoch:02d}-{valid_loss:.4f}-{valid_f1:.4f}',
                                      verbose=False,
                                      mode='min')

trainer = Trainer(
    max_epochs=CFG.num_epochs,
    gpus=1,
    accumulate_grad_batches=CFG.accum,
    precision=CFG.precision,
    callbacks=[EarlyStopping(monitor='valid_loss', patience=3, mode='min')],
    checkpoint_callback=checkpoint_callback,
    enable_checkpointing=True,
    logger=logger,
)

In [ ]:
trainer.fit(lit_model, train_dataloader=train_loader, val_dataloaders=valid_loader)

<a id="metrics"></a>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#648c54;
           font-size:110%;
           font-family:Verdana;
           text-align: center;
           letter-spacing:0.5px">

<h1 style="padding: 10px;
              color:white;"> Plot Metrics
</h1>
</div>

In [ ]:
metrics = pd.read_csv(f'{trainer.logger.log_dir}/metrics.csv')

train_acc = metrics['train_f1'].dropna().reset_index(drop=True)
valid_acc = metrics['valid_f1'].dropna().reset_index(drop=True)
    
fig = plt.figure(figsize=(7, 6))

plt.plot(train_acc, color="r", label='train/f1')
plt.plot(valid_acc, color="b", label='valid/f1')
plt.ylabel('F1', fontsize=24)
plt.xlabel('Epoch', fontsize=24)
plt.legend(loc='lower right', fontsize=18)
plt.savefig(f'{trainer.logger.log_dir}/f1.png')

train_loss = metrics['train_loss'].dropna().reset_index(drop=True)
valid_loss = metrics['valid_loss'].dropna().reset_index(drop=True)

fig = plt.figure(figsize=(7, 6))
plt.plot(train_loss, color="r", label='train/loss')
plt.plot(valid_loss, color="b", label='valid/loss')
plt.ylabel('Loss', fontsize=24)
plt.xlabel('Epoch', fontsize=24)
plt.legend(loc='upper right', fontsize=18)
plt.savefig(f'{trainer.logger.log_dir}/loss.png')\

